In [1]:
import numpy as np
import os

In [2]:
from filtool_script import FiltoolExec

In [3]:
fil_exec = FiltoolExec('/home/rsenzel/Code/inception/dev_injector/fb/controlfb2_uhf.fil', 'pars.json', 'test_dir', 1)

In [4]:
ddplan = list(fil_exec.create_DDplan())
filplan = fil_exec.create_filplan(ddplan)

In [5]:
filplan

'test_dir/filplan.json'

In [6]:
for i in ddplan:
    print(i)

DMRange(low_dm=0.0, high_dm=517.8, dm_step=0.3, tscrunch=1.0)
DMRange(low_dm=517.8, high_dm=834.8, dm_step=0.5, tscrunch=2.0)
DMRange(low_dm=834.8, high_dm=1589.8, dm_step=1.0, tscrunch=4.0)
DMRange(low_dm=1589.8, high_dm=3001.8, dm_step=2.0, tscrunch=8.0)


In [45]:
dm_range = np.linspace(ddplan[0].low_dm, ddplan[0].high_dm, int((ddplan[0].high_dm-ddplan[0].low_dm)/ddplan[0].dm_step), endpoint=False)
dm1 = np.linspace(ddplan[1].low_dm, ddplan[1].high_dm, int((ddplan[1].high_dm-ddplan[1].low_dm)/ddplan[1].dm_step), endpoint=False)

In [63]:
dm_trials_list = []
tscrunch_list = []
for i, dm_range in enumerate(ddplan):
    n_trial = int((dm_range.high_dm-dm_range.low_dm)/dm_range.dm_step)
    n_trial, endpoint = (n_trial + 1, True) if i == len(ddplan)-1 else (n_trial, False)
    dm_list_i = np.linspace(dm_range.low_dm, dm_range.high_dm, n_trial, endpoint=endpoint)
    tscrunch = np.ones_like(dm_list_i) * dm_range.tscrunch
    dm_trials_list.append(dm_list_i)
    tscrunch_list.append(tscrunch)

In [65]:
dm_trials = np.concat(dm_trials_list)
tscrunch = np.concat(tscrunch_list)
tscrunch

array([1., 1., 1., ..., 8., 8., 8.])

In [66]:
def find_n_nearest(dm_arr, tscrunch_arr, target, n):
    nearest_indices = np.argsort(np.abs(dm_arr - target))[:n]
    dm_values = dm_arr[nearest_indices]
    tscrunch_values = tscrunch_arr[nearest_indices]
    return dm_values, tscrunch_values

In [70]:
find_n_nearest(dm_trials, tscrunch, 517.7, 5)

(array([517.8, 517.5, 517.2, 518.3, 516.9]), array([2., 1., 1., 2., 1.]))

In [50]:
my_dms = np.random.uniform(50, 1500, size=10)
nearest = 5
dm_search = np.zeros((len(my_dms), nearest))
my_dms, dm_search

(array([1072.13331736, 1374.67901752, 1121.25312234, 1487.44617915,
        1061.48080809, 1298.42985136, 1143.93419058,  585.98544057,
         226.25915701,  530.1386847 ]),
 array([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]]))